# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT count(*) as transaction_count, sum(total - service_fees) as subtotal, sum(service_fees) as service_fees, sum(total) as total, (created_on)::date, payment_method_code, corp_type_code 
FROM invoices 
WHERE corp_type_code = 'CSO' 
AND invoice_status_code = 'PAID' 
AND payment_method_code in ('PAD','EJV', 'DRAWDOWN') 
AND date(created_on)  > date(current_date - 1 - interval '1 months')
AND date(created_on) <= date(current_date - 1)
GROUP BY (created_on)::date, payment_method_code, corp_type_code;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+'monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:  
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:        
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql monthly_reconciliation_disbursed  <<
SELECT count(*) as transaction_count, sum(total - service_fees) as subtotal, sum(service_fees) as service_fees, sum(total) as total, (updated_on)::date, payment_method_code, corp_type_code
FROM invoices
WHERE corp_type_code = 'CSO'
AND invoice_status_code = 'PAID'
AND payment_method_code in ('PAD','EJV')
AND disbursement_status_code = 'COMPLETED'
AND date(updated_on)  > date(current_date - 1 - interval '1 months')
AND date(updated_on)  <= date(current_date - 1)
GROUP BY (updated_on)::date, payment_method_code, corp_type_code;

Save to another CSV

In [ ]:
filename_disbursed = os.path.join(os.getcwd(), r'data/')+'monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_disbursed = monthly_reconciliation_disbursed.DataFrame()
with open(filename_disbursed, 'a') as f:  
    f.write('Monthly Reconciliation Disbursed:\n\n')
    if df_disbursed.empty:
        f.write('No Data Retrieved')
    else:        
        df_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)